In [2]:
import pandas as pd
import numpy as np
from transformers import pipeline
import re

df = pd.read_excel('/content/Drug_dose.xlsx')
df.head(3)

,Unnamed: 0,drug_name,Age Range,medical_condition_h3,medical_condition_h4,medical_condition_h5,dose_title,dose,dose_range_start,dose_range_end,...,max_dose_from_dose_unit,age_range_start,age_range_end,age_range_unit,body_weight_range_start,body_weight_range_end,body_weight_range_unit,reference,PRN,administration_notes
0,367,dimenhydrinate,Adult,Prevention of Motion Sickness,NaN,NaN,NaN,NaN,50.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"50-100 mg PO/IV/IM q4-6hr PRN, 30 minutes befo...",0.0,NaN
1,367,dimenhydrinate,Adult,Prevention of Motion Sickness,NaN,NaN,NaN,NaN,50.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"50-100 mg PO/IV/IM q4-6hr PRN, 30 minutes befo...",0.0,NaN
2,367,dimenhydrinate,Adult,Prevention of Motion Sickness,NaN,NaN,NaN,NaN,50.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"50-100 mg PO/IV/IM q4-6hr PRN, 30 minutes befo...",0.0,NaN


In [ ]:
df[['age_range_start', 'age_range_end','age_range_unit']].head(5)

,age_range_start,age_range_end,age_range_unit
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN


In [ ]:
set(df.age_range_unit)

{'Infants', 'age', 'days', 'months', nan, 'weeks', 'years'}

In [5]:
df[df.age_range_unit=='Infants'][['age_range_start', 'age_range_end','age_range_unit']]

,age_range_start,age_range_end,age_range_unit
3589,0,NaN,Infants


In [94]:
class AgeRangeEditor:
    def __init__(self, df):
        self.df = df

    def get_unit(self, age_unit):
        if age_unit == 'days':
            return 1
        elif age_unit == 'months':
            return 30
        elif age_unit == 'weeks':
            return 7
        elif str(age_unit) in ['years', 'age','NaN','nan']:
            return 365
        else:
            return 'None'

    def change_number_start(self, age_range_start, value_mul, is_add=False):
        if type(age_range_start) == int:
            if is_add:
                return (age_range_start * value_mul) + 1
            else:
                return age_range_start * value_mul
        return age_range_start

    def change_number_end(self, age_range_end, value_mul, is_sub=False):
        if type(age_range_end) == int:
            if is_sub:
                return (age_range_end * value_mul) - 1
            else:
                return age_range_end * value_mul
        return age_range_end

    def get_diff_day(self, dose_title):
        is_add, is_sub = False, False

        if '>' in str(dose_title):
            is_add = True
        if '<' in str(dose_title):
            is_sub = True
        return is_add, is_sub

    def map_age_range_end(self, age_range_end):
        if type(age_range_end) == int or type(age_range_end) == float:
          bins = [6570, 23720, 54750]
          labels = ['Pediatric', 'Adult', 'Geriatric','other']
          if age_range_end <= bins[0]:
            agerange = labels[0]
          elif age_range_end <= bins[1] and age_range_end > bins[0]:
            agerange = labels[1]
          elif age_range_end <= bins[2] and age_range_end > bins[1]:
            agerange = labels[2]
          else:
            agerange = labels[3]

          return agerange
        return 'other'


    def edit_age_range(self):
        for i in range(len(self.df) ):
            age_unit = self.df['age_range_unit'].iloc[i]
            dose_title = self.df['dose_title'].iloc[i]
            age_range_start = self.df['age_range_start'].iloc[i]
            age_range_end = self.df['age_range_end'].iloc[i]
            value_mul = self.get_unit(age_unit)
            is_add, is_sub = self.get_diff_day(dose_title)

            if value_mul != 'None':
                age_range_start = self.change_number_start(age_range_start, value_mul, is_add)
                age_range_end = self.change_number_end(age_range_end, value_mul, is_sub)
                age_unit = 'days'

            self.df.at[i, 'age_range_start'] = age_range_start

            agerange = self.map_age_range_end(age_range_end)
            if agerange != 'other':
              self.df.at[i,'Age Range'] = agerange
            self.df.at[i, 'age_range_end'] = age_range_end
            self.df.at[i, 'age_range_unit'] = age_unit

class ZeroShotClassifier:
  def __init__(self, model_name="facebook/bart-large-mnli", candidate_labels=None):
    self.classifier = pipeline("zero-shot-classification", model=model_name)
    self.candidate_labels = candidate_labels or []

  def classify_sequence(self, sequence_to_classify):
    result = self.classifier(sequence_to_classify, candidate_labels=self.candidate_labels)
    return result

  def classify_label(self, sequence_to_classify):
    sequence_to_classify = str(sequence_to_classify)
    result = self.classifier(sequence_to_classify, candidate_labels=self.candidate_labels)['labels'][0]
    return result

In [95]:
age_editor = AgeRangeEditor(df)
age_editor.edit_age_range()

# If the age doesn't have a unit, it doesn't require any changes, otherwise it is turned into days as required.
# If the age range could be calculated from the age-range it's done, otherwise it is left as it was.

In [90]:
df[150:]

,Unnamed: 0,drug_name,Age Range,age_range_start,age_range_end,age_range_unit,medical_condition_h3,medical_condition_h4,medical_condition_h5,dose_title,...,duration_unit,maximum_dose,maximum_dose_unit,max_dose_from_dose_unit,body_weight_range_start,body_weight_range_end,body_weight_range_unit,reference,PRN,administration_notes
150,1158,fentanyl,Adult,NaN,NaN,NaN,Analgesia (Off-label),NaN,NaN,Analgesia,...,None,NaN,NaN,NaN,NaN,NaN,NaN,Analgesia: 1-2 mcg/kg IV bolus or 25-100 mcg/d...,NaN,NaN
151,1159,fentanyl,Adult,NaN,NaN,NaN,Analgesia (Off-label),NaN,NaN,Analgesia,...,None,NaN,NaN,NaN,NaN,NaN,NaN,Analgesia: 1-2 mcg/kg IV bolus or 25-100 mcg/d...,NaN,NaN
152,1160,fentanyl,Adult,NaN,NaN,NaN,Analgesia (Off-label),NaN,NaN,Analgesia,...,None,NaN,NaN,NaN,NaN,NaN,NaN,Analgesia: 1-2 mcg/kg IV bolus or 25-100 mcg/d...,NaN,NaN
153,1161,fentanyl,Adult,NaN,NaN,NaN,Analgesia (Off-label),NaN,NaN,Analgesia,...,None,NaN,NaN,NaN,NaN,NaN,NaN,Analgesia: 1-2 mcg/kg IV bolus or 25-100 mcg/d...,NaN,NaN
154,1162,fentanyl,Adult,NaN,NaN,NaN,Analgesia (Off-label),NaN,NaN,Severe pain,...,None,NaN,NaN,NaN,NaN,NaN,NaN,Analgesia: 1-2 mcg/kg IV bolus or 25-100 mcg/d...,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4027,20220,vardenafil,Adult,NaN,NaN,NaN,Erectile Dysfunction,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,10 mg PO ~1 hour before sexual activity; may b...,NaN,NaN
4028,20221,vardenafil,Geriatric,NaN,23725,days,Erectile Dysfunction,NaN,NaN,Film-coated tablet,...,None,NaN,NaN,NaN,NaN,NaN,NaN,<65 years\n\nFilm-coated tablet: 10 mg PO ~1 h...,0.0,NaN
4029,20222,vardenafil,Geriatric,NaN,23725,days,Erectile Dysfunction,NaN,NaN,Orally disintegrating tablet (ODT),...,None,NaN,NaN,NaN,NaN,NaN,NaN,<65 years\n\nFilm-coated tablet: 10 mg PO ~1 h...,0.0,NaN
4030,20223,vardenafil,Geriatric,65,NaN,NaN,Erectile Dysfunction,NaN,NaN,Film-coated tablet,...,None,NaN,NaN,NaN,NaN,NaN,NaN,<65 years\n\nFilm-coated tablet: 10 mg PO ~1 h...,NaN,NaN


In [ ]:
#df.to_csv('/content/Drug_dose_age_days.csv',index=False,encoding='utf-8-sig')

In [76]:
class DurationExtractor:
    def __init__(self, df):
        self.df = df
        self.is_duration = ZeroShotClassifier(candidate_labels=['Talks about a duration', "Does not talk about duration"])

    def contains_number(self, sentence):
        sentence = str(sentence)
        if re.search(r'\d', sentence):
            return sentence
        else:
            return None

    def dont_contain_dose(self, sentence):
        if any(word in str(sentence) for word in ['mg', 'mg/day', 'q2month', 'q2day', 'dose']):
            return False
        return True

    def find_duration_unit(self, duration):
        if type(duration) == int or type(duration) == float:
            return 'Not detected'

        duration = str(duration).lower()
        ls_tars = ['day', 'month', 'week', 'minute', 'hr', 'cycle', '24hr', 'hr']
        if any(word in duration for word in ls_tars):
            for word in ls_tars:
                if word in duration:
                    return word
        return 'Not detected'

    def preprocess_duration(self, duration):
        ls_tars = ['minimum of ', 'minimum ', '≤', 'up to ', 'at least ']
        if any(word in str(duration) for word in ls_tars):
            for word in ls_tars:
                duration = duration.replace(word, '0-')
        return duration

    def extract_from_duration(self, duration):
        if type(duration) == int or type(duration) == float:
            return [0, duration]

        duration = str(str(duration).split('or ')[0])
        numbers = re.findall(r'\d+', duration)

        if len(numbers) == 1:
            end_duration, start_duration = numbers[0], 0

        elif len(numbers) > 1:
            end_duration, start_duration = numbers[1], numbers[0]

        else:
            end_duration, start_duration = 0, 0
        return [int(start_duration), int(end_duration)]

    def get_duration_output(self, duration):
        if str(duration) != 'nan':
            lab_out = self.is_duration.classify_label(duration)
        else:
            lab_out = 'Talks about nothing'
        start_duration, end_duration, duration_unit = 'None', 'None', 'None'

        if lab_out == 'Talks about a duration':
            if self.contains_number(duration) and self.dont_contain_dose(duration):
                duration = self.preprocess_duration(duration)
                duration_unit = self.find_duration_unit(duration)
                start_duration, end_duration = self.extract_from_duration(duration)
        return start_duration, end_duration, duration_unit

    def process_duration_column(self):
        duration_data = self.df['duration'].apply(self.get_duration_output)
        start_durations, end_durations, duration_units = zip(*duration_data)

        self.df['duration_start'] = start_durations
        self.df['duration_end'] = end_durations
        self.df['duration_unit'] = duration_units


In [77]:
duration_extractor = DurationExtractor(df)
duration_extractor.process_duration_column()

In [80]:
df

,Unnamed: 0,drug_name,Age Range,medical_condition_h3,medical_condition_h4,medical_condition_h5,dose_title,dose,dose_range_start,dose_range_end,...,age_range_end,age_range_unit,body_weight_range_start,body_weight_range_end,body_weight_range_unit,reference,PRN,administration_notes,duration_start,duration_end
0,367,dimenhydrinate,Adult,Prevention of Motion Sickness,NaN,NaN,NaN,NaN,50.0,100.0,...,NaN,NaN,NaN,NaN,NaN,"50-100 mg PO/IV/IM q4-6hr PRN, 30 minutes befo...",0.0,NaN,None,None
1,367,dimenhydrinate,Adult,Prevention of Motion Sickness,NaN,NaN,NaN,NaN,50.0,100.0,...,NaN,NaN,NaN,NaN,NaN,"50-100 mg PO/IV/IM q4-6hr PRN, 30 minutes befo...",0.0,NaN,None,None
2,367,dimenhydrinate,Adult,Prevention of Motion Sickness,NaN,NaN,NaN,NaN,50.0,100.0,...,NaN,NaN,NaN,NaN,NaN,"50-100 mg PO/IV/IM q4-6hr PRN, 30 minutes befo...",0.0,NaN,None,None
3,368,dimenhydrinate,Adult,Meniere's Disease (Off-label),NaN,NaN,NaN,NaN,25.0,50.0,...,NaN,NaN,NaN,NaN,NaN,25-50 mg PO q8hr for maintenance\n\n50 mg IM f...,0.0,NaN,None,None
4,369,dimenhydrinate,Adult,Meniere's Disease (Off-label),NaN,NaN,NaN,50,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,25-50 mg PO q8hr for maintenance\n\n50 mg IM f...,0.0,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4027,20220,vardenafil,Adult,Erectile Dysfunction,NaN,NaN,NaN,10,5.0,20.0,...,NaN,NaN,NaN,NaN,NaN,10 mg PO ~1 hour before sexual activity; may b...,NaN,NaN,None,None
4028,20221,vardenafil,Geriatric,Erectile Dysfunction,NaN,NaN,Film-coated tablet,10,5.0,20.0,...,65,years,NaN,NaN,NaN,<65 years\n\nFilm-coated tablet: 10 mg PO ~1 h...,0.0,NaN,None,None
4029,20222,vardenafil,Geriatric,Erectile Dysfunction,NaN,NaN,Orally disintegrating tablet (ODT),10,NaN,NaN,...,65,years,NaN,NaN,NaN,<65 years\n\nFilm-coated tablet: 10 mg PO ~1 h...,0.0,NaN,None,None
4030,20223,vardenafil,Geriatric,Erectile Dysfunction,NaN,NaN,Film-coated tablet,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,<65 years\n\nFilm-coated tablet: 10 mg PO ~1 h...,NaN,NaN,None,None


In [85]:
df= df[['Unnamed: 0', 'drug_name', 'Age Range','age_range_start',
       'age_range_end', 'age_range_unit', 'medical_condition_h3',
       'medical_condition_h4', 'medical_condition_h5', 'dose_title', 'dose',
       'dose_range_start', 'dose_range_end', 'dose_unit', 'is_divided',
       'route_of_administration', 'infusion_rate', 'frequency',
       'freq_from_dose_unit', 'frequency_range_start', 'frequency_range_end',
       'frequency_unit', 'duration', 'duration_start', 'duration_end', 'duration_unit',
        'maximum_dose','maximum_dose_unit', 'max_dose_from_dose_unit',
       'body_weight_range_start', 'body_weight_range_end', 'body_weight_range_unit',
        'reference', 'PRN', 'administration_notes']]

In [97]:
df[['Age Range','age_range_start', 'age_range_end', 'age_range_unit','duration', 'duration_start', 'duration_end', 'duration_unit']][250:300]

,Age Range,age_range_start,age_range_end,age_range_unit,duration,duration_start,duration_end,duration_unit
250,Adult,NaN,NaN,days,3,0,3,Not detected
251,Adult,NaN,NaN,days,NaN,None,None,None
252,Adult,NaN,NaN,days,3,0,3,Not detected
253,Adult,NaN,NaN,days,5,0,5,Not detected
254,Adult,NaN,NaN,days,NaN,None,None,None
255,Adult,NaN,NaN,days,5,0,5,Not detected
256,Pediatric,730,NaN,days,NaN,None,None,None
257,Pediatric,730,NaN,days,3,0,3,Not detected
258,Pediatric,365,NaN,days,NaN,None,None,None
259,Pediatric,730,NaN,days,3 consecutive days,0,3,day


In [93]:
str(df.age_range_unit.iloc[4030])

'nan'

In [98]:
df.to_csv('/content/Drug_dose_age_duration_v1.csv',index=False,encoding='utf-8-sig')